## Preamble
* Increase notebook cell width: `<style>.container { width:86% !important; }</style>`
* Convert specific cells to HTML only:
`jupyter nbconvert --to html --no-input my-notebook.ipynb`  
`jupyter nbconvert --to html --TagRemovePreprocessor.remove_cell_tags code my-notebook.ipynb`

* Do not use `fig.write_html('...')` it will return `UnicodeEncodeError`. Use 
  ```
  with open(path, "w", encoding="utf-8") as f: 
      f.write(fig.to_html()
  ```
  instead
* For development, `fig.update_layout(width=600, height=400)` and `fig.show()`
* For production , save it. Then use HTML:
    ```
    %%HTML
    <table>
      <tr>
        <td>
        <iframe width=560 height=400 src="./..." frameborder="0"></iframe>
        </td>
        <td>
        <iframe width=560 height=400 src="./..." frameborder="0"></iframe>
        </td>
      </tr>
    </table>
    ```

In [75]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from IPython.display import IFrame

def _plot_comparison(model, x, y, return_fig=True, color='grey'):
    def sgn(x):
        if x > 0: return f' + {x}'
        else: return f' - {abs(x)}'
        
    a, b, c = np.polyfit(x, y, deg=2)
    d, e = np.polyfit(x, y, deg=1)
    if return_fig:
        X = np.linspace(np.min(x), np.max(x), num=100)
        Y = a*X**2 + b*X + c
        Z = d*X + e
        
        fig1 = go.Scatter(x=x, y=y, mode='markers',
                          text=[f'{model}<br>{int(1000*battery)}Wh / {int(rang)}km' \
                                for model, battery, rang in zip(model, x, y)],
                          marker=dict(size=10, color='blue'), 
                          marker_color=color, showlegend=False)
        fig2 = go.Scatter(x=X, y=Y, mode='lines', 
                          name=f'y = {round(a, 2)}x²{sgn(round(b, 2))}x{sgn(round(c, 2))}',
                          line_color='black')
        fig3 = go.Scatter(x=X, y=Z, mode='lines', 
                          name=f'y = {round(d, 2)}x{sgn(round(e, 2))}', fill='tonexty',
                          line_color='rgb(200, 200, 200)')

        return fig1, fig2, fig3
    else:
        return [a, b, c]

def plot_capacity(df, brand='', drop=None, return_fig=True, color='grey'):
    _df = df.copy()
    _df['color'] = color
    if brand != '':
        _df = _df[_df['brand'] == brand]
    if drop is not None:
        _df = _df[~_df['model'].isin(drop)]
    return _plot_comparison(_df['model'], _df['battery']/1000, _df['range'], 
                            return_fig=return_fig, color=_df['color'])


def plot_Kcapacity(df, brand='', rider=60, drop=None, return_fig=True, color='grey'):
    _df = df.copy()
    _df['color'] = color
    
    if brand != '':
        _df = _df[_df['brand'] == brand]
    if drop is not None:
        _df = _df[~_df['model'].isin(drop)]
    K = (rider + _df['weight'])/rider
    return _plot_comparison(_df['model'], _df['battery']/1000, K * _df['range'], 
                            return_fig=return_fig, color=_df['color'])

def find_weight(df, brand='Inmotion', values=list(range(-500, 0)) + list(range(1,500)), limitter=(-20, 20)):
    d = dict()
    for i in values:
        a, b, c = plot_Kcapacity(df, brand=brand, rider=i, return_fig=False)
        if limitter[0] <= a <= limitter[1]:
            d[i] = a
    fig1 = go.Scatter(x=list(d.keys()), y=list(d.values()), mode='markers',
                          text=[f'Weight: {x}; a: {y}' for x, y in d.items()],
                          marker=dict(size=3, color='blue'), name='hj')
    fig = go.Figure(data=[fig1])
    fig.update_layout(xaxis_title="Theoretical riders weight", 
                      yaxis_title='Value of a in line of approximation (y=ax²+bx+c)', 
                      width=1000)
    return fig

def compare_model_capacity(df, name='', second_name='', drop=None, color='grey'):
    fig = make_subplots(rows=1, cols=1)
    c1, c2, c3 = plot_capacity(df, brand=name, drop=drop, color=color)
    fig.add_trace(c2, row=1, col=1)
    fig.add_trace(c3, row=1, col=1)
    fig.add_trace(c1, row=1, col=1)
    fig.update_xaxes(title_text="Battery capacity, kWh", row=1, col=1)
    fig.update_yaxes(title_text="Range, km", row=1, col=1)
    fig.update_layout(height=600, width=1000, title_text=f"Battery vs Declared range of {name} EUC {second_name}")
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0.0, dtick = 0.2),
                      yaxis = dict(tickmode = 'linear', tick0 = 0.0, dtick = 25))
    fig.update_traces(marker=dict(size=10, line=dict(width=2, color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    return fig
    
def compare_model_Kcapacity(df, rider=60, name='', second_name='', drop=None, color='grey'):
    fig = make_subplots(rows=1, cols=1)
    kc1, kc2, kc3 = plot_Kcapacity(df, brand=name, rider=rider, drop=drop, color=color)
    fig.add_trace(kc2, row=1, col=1)
    fig.add_trace(kc3, row=1, col=1)
    fig.add_trace(kc1, row=1, col=1)
    fig.update_xaxes(title_text="Battery capacity, kWh", row=1, col=1)
    fig.update_yaxes(title_text=f"(1 + W/{rider}) x Range, km", row=1, col=1)
    fig.update_layout(height=600, width=1000, title_text=f"Battery vs Theoretic range of {name} EUC {second_name}")
    fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0.0, dtick = 0.2), 
                      yaxis = dict(tickmode = 'linear', tick0 = 0.0, dtick = 25))
    fig.update_traces(marker=dict(size=10, line=dict(width=2, color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    return fig
    
df = pd.read_csv('euc_data_viensrats.txt')
euc_range_research_htmlplots = 'euc_range_research_plots'
if euc_range_research_htmlplots not in os.listdir():
    os.mkdir(euc_range_research_htmlplots)

# What's the real range of EUC?
A lot of manufacturers gives one but sometimes they are just too high to be realistic. We need an another way to indicate it.

## Goals:
* Is there any way to predict a real EUC range by a data provided by manufacturers?
* Are there manufacturers providing ranges of their EUC not accurately?
* How do the newest models contributes to results of predicted ranges?
* Any improvemens of prediction model?

## Data
I use data that is provided in [https://viensrats.lv/product-category/unicycle/](https://viensrats.lv/product-category/unicycle/), a Latvian shop of EUC.

In [9]:
df

,brand,model,speed,weight,range,wheel,battery,power
0,Gotway,Gotway MCM5 V2 460wh 84V,34.0,16.0,30.0,14.0,340.0,1500.0
1,Gotway,Begode/Gotway RS 1800Wh 100v,70.0,27.0,160.0,18.0,1800.0,2600.0
2,Gotway,Begode/Gotway EX.N 19″ 2700Wh 100v,70.0,33.0,180.0,20.0,2700.0,2800.0
3,Gotway,Begode (Extreme Bull) Commander 20” 3600 Wh 100v,80.0,38.0,230.0,19.0,3600.0,3000.0
4,Gotway,Begode Mten4 750Wh,57.0,13.0,70.0,11.0,750.0,1000.0
5,Gotway,Gotway Nikola Plus 1800Wh,60.0,25.0,170.0,17.0,1776.0,2000.0
6,Gotway,Gotway MCM5 800Wh V2,40.0,16.9,70.0,14.0,777.0,1500.0
7,Gotway,Begode(Gotway) Tesla V3 1500Wh 84V,50.0,22.0,120.0,16.0,1500.0,2000.0
8,Gotway,Begode Master 2400Wh 134.4V V2,80.0,36.0,180.0,19.0,2400.0,3500.0
9,Gotway,Extreme Bull (Begode) X-men Yellow HT 1800 Wh ...,75.0,31.0,140.0,19.0,1800.0,2800.0


## Insight

How will battery capacity affect a range of EUC? Let's check a Gotway brand. At the January of 2023 it has had 23 brands.

In the first diagram, a rate of range and capacity tends to decrease as batteries get bigger. The point is, the more heavier EUC, the more load goes for motor and more power is consumed. A load contains not only rider's weight but also a weight of EUC. Majority of manufacturers provides a realistic range that involves both rider's weight (70 kilos usually) and EUC weight. As seen in a diagram, there's no proportion between realistic range and battery capacity.

Taking this into account, a second diagram provides values of realistic ranges converted into theoretic ones assuming EUC is weightless. It seems reasonable that if load gets $k$ times smaller then we should expect $k$ times larger range:

$$k = (W + w)/W\text{ where: }\begin{cases}w\text{ is the weight of EUC} \\ W \text{ is the rider's weight}\end{cases}$$

So:

$$\text{Theoretic range} = k \times \text{range provided by manufacturer}$$

Indeed, in a second diagram proportion is clear.

In [76]:
fig = compare_model_capacity(df, name = 'Gotway', color='purple')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/gotway_cmp1.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())
            
fig = compare_model_Kcapacity(df, rider=70, name = 'Gotway', second_name = '(rider = 70kg)', color='purple')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/gotway_cmp2.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())

In [98]:
%%HTML
<style>.container { width:85% !important; }</style>
<table>
  <tr>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/gotway_cmp1.html" frameborder="0"></iframe>
    </td>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/gotway_cmp2.html" frameborder="0"></iframe>
    </td>
  </tr>
</table>

,


In [109]:
fig = compare_model_capacity(df, name='Kingsong', color='red')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/kingsong_cmp1.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())
            
fig = compare_model_Kcapacity(df, rider=70, name='Kingsong', second_name = '(rider = 70kg)', color='red')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/kingsong_cmp2.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())

However, the same is not straightforward for Kingsongs (12 models) in total. Declared range seems even more accurate than theoretic one:

In [110]:
%%HTML
<style>.container { width:85% !important; }</style>
<table>
  <tr>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/kingsong_cmp1.html" frameborder="0"></iframe>
    </td>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/kingsong_cmp2.html" frameborder="0"></iframe>
    </td>
  </tr>
</table>

,


In [111]:
fig = compare_model_capacity(df, name='Inmotion', color='green')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/inmotion_cmp1.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())
            
fig = compare_model_Kcapacity(df, rider=70, name='Inmotion', second_name = '(rider = 70kg)', color='green')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/inmotion_cmp2.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())

And for 8 models of Inmotion it's better:

In [112]:
%%HTML
<style>.container { width:85% !important; }</style>
<table>
  <tr>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/inmotion_cmp1.html" frameborder="0"></iframe>
    </td>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/inmotion_cmp2.html" frameborder="0"></iframe>
    </td>
  </tr>
</table>

,


### How do the newest models contributes to the results of predicted ranges?

2022 brought us a huge progress in EUC World:
  * Begode/Gotway released Begode Master PRO 4800Wh
  * Kingsong released Kingsong S22 Eagle 2220Wh
  * Inmotion released Inmotion v13 Challenger 3024Wh
    
Check to see how it affects the results of range prediction:

In [115]:
fig = compare_model_Kcapacity(df, rider=70, name = 'Gotway', 
                        second_name = '(with no Master PRO)', 
                        drop = ['Begode Master PRO 4800wh 134V'], color='purple')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/gotway_w.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())
    
fig = compare_model_Kcapacity(df, rider=70, name = 'Kingsong', 
                        second_name = '(with no KS22)',
                        drop=['Kingsong S22 Eagle'], color='red')
path = f'euc_range_research_plots/kingsong_w.html'
fig.update_layout(width=600, height=400)
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())
    
fig = compare_model_Kcapacity(df, rider=70, name = 'Inmotion', 
                              second_name = '(with no V13)', 
                              drop=['Inmotion v13 Challenger'], color='green')
fig.update_layout(width=600, height=400)
path = f'euc_range_research_plots/inmotion_w.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())

In [117]:
%%HTML
<style>.container { width:85% !important; }</style>
<table>
  <tr>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/gotway_cmp2.html" frameborder="0"></iframe>
    </td>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/gotway_w.html" frameborder="0"></iframe>
    </td>
  </tr>
  <tr>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/kingsong_cmp2.html" frameborder="0"></iframe>
    </td>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/kingsong_w.html" frameborder="0"></iframe>
    </td>
  </tr>
  <tr>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/inmotion_cmp2.html" frameborder="0"></iframe>
    </td>
    <td>
    <iframe width=560 height=400 src="./euc_range_research_plots/inmotion_w.html" frameborder="0"></iframe>
    </td>
  </tr>
</table>

,
,
,


  * Release of Inmotion V13 increases a correlation between battery and theoretical range signifantly. Ranges of other current models of Inmotion (especialy V12 HS and V12 HT) have been underestimated until V13 came.
  * Release of Begode Master Pro stabilizes a trend as well. Though scenario is different. Current models of Begode/Gotway promises slightly less range than we could expect from a trend of older ones.
  * In Kingsong world, correlation have been high until KS22 came. It looks like KS22 manufacturers provides range that's is too high to fit with a trend of previous models.

In [119]:
c = {'Kingsong':'red', 'Inmotion':'green', 'Gotway':'purple', 'Leaperkim':'orange', 'Ninebot':'yellow'}
fig = compare_model_Kcapacity(df, rider=60, second_name = '(assuming rider = 60kg)',
                        drop=None, color=[c[n] for n in df['brand']])

fig.update_layout(width=850, height=600)
path = f'euc_range_research_plots/allin.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())

Putting all the models together

In [121]:
%%HTML
<iframe width=850 height=600 src="./euc_range_research_plots/allin.html" frameborder="0"></iframe>

## Overall



Let $$k = (W + w)/W = 1 + w/70 \text{ where: }\begin{cases}w\text{ is the weight of EUC} \\ W \text{ is the rider's weight, usually 70}\end{cases}$$

and $$\begin{cases} x = \text{battery capacity} \\ w = \text{weight of EUC}\end{cases}$$ 
Then $r$ could be estimated by formula: $$r = \frac{111.72x + 7.58}{k} = \frac{111.72x + 7.58}{1 + w/60}$$ or slightly more accurate:

$$r = \frac{-4.25x^2 + 129.29x - 4.33}{k} = \frac{-4.25x^2 + 129.29x - 4.33}{1 + w/70}$$

Similar estimations for theoretic range of specific brands:
    
|Brand|Linear approximation|quadratic approximation|
|:---|:---|:---|
|Gotway| $$109.543x + 11.606$$ | $$-1.045x² + 114.348x + 7.594$$
|Inmotion| $$125.69x - 6.168$$ | $$-1.297x² + 129.915x - 8.603$$
|Kingsong| $$116.563x - 1.539$$ | $$2.173x² + 113.128x - 0.572$$

Comparison of treds of different brands:

In [125]:
def draw_approx(name, P1, P2, color='grey'):
    def sgn(x):
        if x > 0: return f' + {x}'
        else: return f' - {abs(x)}'
    a, b, c = P1
    d, e = P2
    X = np.linspace(0, 4, num=200)
    Y = a*X**2 + b*X + c
    Z = d*X + e

    fig1 = go.Scatter(x=X, y=Y, mode='lines', 
                      name=f'{name}: {round(a, 2)}x²{sgn(round(b, 2))}x{sgn(round(c, 2))}',
                      line_color=color)
    fig2 = go.Scatter(x=X, y=Z, mode='lines', 
                      name=f'{name}: {round(d, 2)}x{sgn(round(e, 2))}', fill='tonexty',
                      line_color=color)

    return fig1, fig2

fig1, fig2 = draw_approx('Gotway', [-1.045, 114.348, 7.594], [109.543, 11.606], color='purple')
fig3, fig4 = draw_approx('Inmotion', [-1.297, 129.915, -8.603], [125.69, -6.168], color='green')
fig5, fig6 = draw_approx('Kingsong', [2.173, 113.128, -0.572], [116.563, -1.539], color='red')
fig = go.Figure(data=[fig1, fig2, fig3, fig4, fig5, fig6])
fig.update_layout(xaxis_title="Battery capacity", 
                  yaxis_title='Approximation or (1 + w/60) x range', 
                  width=1000)
fig.update_layout(width=1000, height=600)
path = f'euc_range_research_plots/allin_trends.html'
with open(path, "w", encoding="utf-8") as f: 
    f.write(fig.to_html())

In [126]:
%%HTML
<iframe width=1000 height=600 src="./euc_range_research_plots/allin_trends.html" frameborder="0"></iframe>

In [145]:
range_df = df[['brand', 'model', 'range']].copy()
x = (df['battery']/1000).copy()
k = 1 + df['weight']/70
range_df['expected range, overall'] = ((-4.25*x*x + 129.29*x - 4.33)/k).round(2)
range_df['expected range, brand-specific'] = range_df['expected range, overall']
range_df.loc[range_df['brand'] == 'Gotway', 'expected range, brand-specific'] = ((-1.045*x*x + 114.348*x + 7.594)/k).round(2)
range_df.loc[range_df['brand'] == 'Inmotion', 'expected range, brand-specific'] = ((-1.297*x*x + 129.915*x - 8.603)/k).round(2)
range_df.loc[range_df['brand'] == 'Kingsong', 'expected range, brand-specific'] = ((2.173*x*x + 113.128*x - 0.572)/k).round(2)
range_df.to_csv('EUC_ranges.txt', index=False)
range_df

,brand,model,range,"expected range, overall","expected range, brand-specific"
0,Gotway,Gotway MCM5 V2 460wh 84V,30.0,31.86,37.73
1,Gotway,Begode/Gotway RS 1800Wh 100v,160.0,154.88,151.57
2,Gotway,Begode/Gotway EX.N 19″ 2700Wh 100v,180.0,213.24,209.81
3,Gotway,Begode (Extreme Bull) Commander 20” 3600 Wh 100v,230.0,263.17,262.96
4,Gotway,Begode Mten4 750Wh,70.0,76.11,78.24
5,Gotway,Gotway Nikola Plus 1800Wh,170.0,156.12,152.81
6,Gotway,Gotway MCM5 800Wh V2,70.0,75.37,77.18
7,Gotway,Begode(Gotway) Tesla V3 1500Wh 84V,120.0,136.99,134.49
8,Gotway,Begode Master 2400Wh 134.4V V2,180.0,185.89,182.27
9,Gotway,Extreme Bull (Begode) X-men Yellow HT 1800 Wh ...,140.0,148.75,145.57


## Free data

Manufacturer provided characteristics of 47 brands: [`.txt` file](euc_data_viensrats.txt)  
Table of ranges derived here: [`.txt` file](EUC_ranges.txt)

It could also be accessed as a plain [`.txt` file](EUC_ranges.txt)

More [discussion on electricunicycle forum](https://forum.electricunicycle.org/topic/30297-best-way-to-estimate-expected-range-of-eucs-by-model-name-driving-conditions/#comment-439411)